In [1]:
# Install required packages (uncomment if needed)
# !pip install earthengine-api geemap

import ee
import geemap
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

In [2]:
# Authenticate and initialise Earth Engine
ee.Authenticate()
ee.Initialize()

In [19]:
# Bounding box in WGS84 [xmin, ymin, xmax, ymax]
bbox = [-44.07166500771045, 81.67474037410186, -43.87093088605835, 81.72574342759195]
region = ee.Geometry.Rectangle(bbox)

# Output directory
repo_dir = os.getcwd()
out_dir = os.path.join(repo_dir, 'Data', 'Landsat')
temp_dir = os.path.join(repo_dir, 'Data', 'Landsat', 'temp')
os.makedirs(out_dir, exist_ok=True)
os.makedirs(temp_dir, exist_ok=True)

# Define date ranges
dates = {
    "1990": ("1990-06-16", "1990-08-01", "1990-08-03", "1990-08-17"),
    "2014": ("2014-06-28", "2014-07-07", "2014-08-15", "2014-08-26")
}

# Bounding box region
bbox = [-44.18926, 81.6518, -43.70383, 81.77820]
region = ee.Geometry.Rectangle(bbox)


In [20]:
def download_landsat_rgb(image_id, out_name, region):
    """
    Downloads a Landsat image, selects RGB bands, clips it to a region, 
    visualises it in a notebook map, and exports it locally as a GeoTIFF.

    Parameters:
        image_id (str): Earth Engine Landsat image asset ID.
        out_name (str): Base filename for saving the output file.
        region (ee.Geometry): The region to clip and export.

    Returns:
        str: File path to the exported GeoTIFF.
    """

    image = ee.Image(image_id).select(rgb_bands).clip(region)
    
    # # Display in notebook (optional)
    # Map = geemap.Map()
    # Map.centerObject(region, 10)
    # Map.addLayer(image, vis_params, f'{out_name} RGB')
    # Map.addLayer(region, {}, "Bounding Box")
    # display(Map)
    
    # Export to local GeoTIFF
    export_path = os.path.join(out_dir, f'{out_name}.tif')
    geemap.ee_export_image(
        image,
        filename=export_path,
        region=region,
        scale=30,
        file_per_band=False
    )
    print(f"Image downloaded: {export_path}")
    return export_path

In [21]:
def reproject_raster(input_path, epsg=3413):
    """
    Reprojects a GeoTIFF raster to a specified EPSG coordinate system.

    Parameters:
        input_path (str): File path to the input raster.
        epsg (int): EPSG code for the target CRS (default = 3413 for NSIDC Polar Stereographic North).

    Returns:
        str: File path to the reprojected output raster.
    """
    filename = os.path.basename(input_path)
    output_path = os.path.join(out_dir, filename.replace('.tif', f'_EPSG{epsg}.tif'))
    target_crs = CRS.from_epsg(epsg)

    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )
    
    print(f"Reprojection complete: {output_path}")
    return output_path

In [22]:
# Landsat image IDs and export names
image_info_2014 = [
    {"id": 'LANDSAT/LC08/C02/T1_L2/LC08_038245_20140628', "out_name": "LC08_20140628"},
    {"id": 'LANDSAT/LC08/C02/T1_L2/LC08_037245_20140707', "out_name": "LC08_20140707"},
    {"id": 'LANDSAT/LC08/C02/T1_L2/LC08_038245_20140815', "out_name": "LC08_20140815"},
    {"id": 'LANDSAT/LC08/C02/T1_L2/LC08_019248_20140826', "out_name": "LC08_20140826"},
]

# RGB bands (SR_B4 = red, SR_B3 = green, SR_B2 = blue)
rgb_bands = ['SR_B4', 'SR_B3', 'SR_B2']

# Visualization stretch for display
vis_params = {
    'bands': rgb_bands,
    'min': 5000,
    'max': 15000,
    'gamma': 1.4
}


In [23]:
for img in image_info_2014:
    tif_path = download_landsat_rgb(img["id"], img["out_name"], region)
    reproject_raster(tif_path)

Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LC08_20140628.tif
Image downloaded: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LC08_20140628.tif
Reprojection complete: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LC08_20140628_EPSG3413.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LC08_20140707.tif
Image downloaded: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LC08_20140707.tif
Reprojection complete: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LC08_20140707_EPSG3413.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures

In [24]:
# Landsat image IDs and export names
image_info_1990 = [
    {"id": 'LANDSAT/LT05/C02/T1_L2/LT05_032001_19900616', "out_name": "LT05_19900616"},
    {"id": 'LANDSAT/LT05/C02/T1_L2/LT05_018248_19900801', "out_name": "LT05_19900801"},
    {"id": 'LANDSAT/LT05/C02/T1_L2/LT05_032001_19900803', "out_name": "LT05_19900803"},
    {"id": 'LANDSAT/LT05/C02/T1_L2/LT05_018248_19900817', "out_name": "LT05_19900817"},
]

# RGB bands (SR_B3 = red, SR_B2 = green, SR_B1 = blue)
rgb_bands = ['SR_B4', 'SR_B3', 'SR_B2']

# Visualization stretch for display
vis_params_l5 = {'min': 0.05, 'max': 0.35, 'gamma': 1.2}



In [25]:
for img in image_info_1990:
    tif_path = download_landsat_rgb(img["id"], img["out_name"], region)
    reproject_raster(tif_path)

Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LT05_19900616.tif
Image downloaded: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LT05_19900616.tif
Reprojection complete: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LT05_19900616_EPSG3413.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LT05_19900801.tif
Image downloaded: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LT05_19900801.tif
Reprojection complete: /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures/Data/Landsat/LT05_19900801_EPSG3413.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Python Coding/Harder-glacier-subglacial-outburst/Figures